<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/medical2.png?raw=true" width=800 align="left"/>

# 실습내용

- 학습용 데이터와 평가용 데이터가 별도의 파일로 제공되는 경우가 있습니다.
- 이미 우리가 학습한 방법과는 조금 다른 접근이 필요합니다.
- 의료비 예측을 의뢰한 가상 고객의 요청을 처리해 봅니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 학습용 데이터 불러오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/insurance_train.csv'
data1 = pd.read_csv(path)

In [ ]:
# 평가용 데이터 불러오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/insurance_test.csv'
data2 = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.
- 모델링에 집중하기 위해 본 실습에서는 탐색 과정을 생략합니다.

In [ ]:
# 학습용 데이터 확인
data1

In [ ]:
# 평가용 데이터 확인
data2

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 가변수화**

In [ ]:
# 가변수화
dumm_cols = ['sex', 'smoker', 'region']
data1 = pd.get_dummies(data1, columns=dumm_cols, drop_first=True)

# 확인
data1.head()

**2) x, y 분리**

In [ ]:
# x, y 분리
target = 'charges' 
x = data1.drop(target, axis=1)
y = data1.loc[:, target]

**3) 학습용, 평가용 데이터 분리**

In [ ]:
# 학습용, 검증용 분리
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 우선 회귀 문제인지 분류 문제인지 명확히 구분합니다.

In [ ]:
# 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# 선언하기
model = DecisionTreeRegressor(max_depth=5, random_state=1)

In [ ]:
# 학습하기
model.fit(x_train, y_train)

In [ ]:
# 예측하기
y_val_pred = model.predict(x_val)

In [ ]:
# 평가하기
print('MAE:', mean_absolute_error(y_val, y_val_pred))
print('R2:', r2_score(y_val, y_val_pred))   

# 5.일반화된 성능

- 과연 앞에서 만든 모델의 일반화된 성능은 어떻게 될까요?
- 즉, 고객사에 어느 정도의 성능이 예상된다고 말할 수 있을까요?
- K-Fold CV로 일반화된 성능을 예측합니다.

In [ ]:
# 불러오기
from sklearn.model_selection import cross_val_score

# 성능예측
cv_score = cross_val_score(model, x_train, y_train, cv=5)

# 결과
print(cv_score)
print('평균:', cv_score.mean())

# 6. 성능 튜닝

- 앞에서 max_depth 값을 5로 고정함이 아쉽습니다.
- Grid Search로 최적의 하이퍼파라미터를 찾아봅니다.

In [ ]:
# 불러오기
from sklearn.model_selection import GridSearchCV

# 기본 모델 선언
model_dt = DecisionTreeRegressor(random_state=1)

# 파라미터 선언
params = {'max_depth': range(1, 51)}

# 모델 선언
model = GridSearchCV(model_dt,
                     params,
                     cv=5, 
                     scoring='r2')                     

In [ ]:
# 학습하기
model.fit(x_train, y_train)

In [ ]:
# 예측 결과 확인
print(model.best_params_)
print(model.best_score_) 

In [ ]:
# 성능 검증
y_val_pred = model.predict(x_val)
print(r2_score(y_val, y_val_pred)) 

# 7.최종 평가

- 최적의 하이퍼파라미터로 학습된 모델로 예측합니다.
- 평가 데이터에 대해서도 필요한 전처리를 수행합니다.

In [ ]:
# 평가 데이터 확인
data2.head()

In [ ]:
# 평가 데이터 가변수화
dumm_cols = ['sex', 'smoker', 'region']
data2 = pd.get_dummies(data2, columns=dumm_cols, drop_first=True)

# 확인
data2.head()

In [ ]:
# 예측하기
x_test = data2
y_pred = model.predict(x_test)

In [ ]:
# 확인
y_pred[:10]

# 8. 파일 제출

- 모델로 예측한 값을 파일에 붙여 엑셀 파일로 저장해 고객사에 제출합니다.

In [ ]:
# 파일 다시 불러오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/insurance_test.csv'
final = pd.read_csv(path)

# 예측결과 추가
final['charges'] = y_pred.round(4)

# 확인
final

In [ ]:
# 엑셀로 저장
final.to_excel('InsurancePred.xlsx')

# 9.정리

- 실제 의료비가 지출된 경우 우리 모델의 성능이 확인될 것입니다.
- 우리는 학습용 데이터로 나름 최선을 다했습니다.
- 그래도 뭔가 아쉬움이 남지요? 
- 성능이 더 좋은 알고리즘을 배워서 사용할 수 있다면....